# **STEP 13:** Extracting Features

In [1]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import numpy as np
import pandas as pd
import SimpleITK as sitk
import radiomics
from radiomics import featureextractor
import os
import logging
import csv
from collections import OrderedDict

In [2]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='testLog_Step13.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

#### **Atenção**: Mudar label do params.yaml 1 para Isodose Volume e 255 para Individual Tumor Mask)

### Extract Features from MRI Treatment using Individual ROI Mask

In [27]:
# Extract Features from MRI Treatmtn Image using Individual ROI Mask
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_ROI_Individual.csv'

def resample_mask_to_image(mask_path, image_path, binary_value=255):
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path))

    mask_resampled = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, mask.GetPixelID()
    )

    mask_np = sitk.GetArrayFromImage(mask_resampled)
    mask_np[mask_np != 0] = binary_value  # usa 255 por defeito; podes mudar para 1
    mask_resampled = sitk.GetImageFromArray(mask_np)

    mask_resampled.SetOrigin(image.GetOrigin())
    mask_resampled.SetSpacing(image.GetSpacing())
    mask_resampled.SetDirection(image.GetDirection())
    return mask_resampled, image

def resolve_registered_volume(mr_dir: Path, fname: str) -> Path | None:
    # if in the CSV the volume is only with Registered, it goes to grab the first registered image from the treatment folder
    mr_dir = Path(str(mr_dir).strip())
    fname  = str(fname).strip()
    direct = mr_dir / fname
    if direct.exists():
        return direct

    # fallback to *_Registered_1.nii.gz
    if fname.endswith("Registered.nii.gz"):
        alt = mr_dir / fname.replace("Registered.nii.gz", "Registered_1.nii.gz")
        if alt.exists():
            return alt

    # any *_Registered_*.nii.gz in case there is no *_Registered_1.nii.gz
    candidates = sorted(mr_dir.glob("*Registered_*.nii.gz"))
    if candidates:
        return candidates[0]
    return None

def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    skip_words = ("skull", "outer contour", "outer-contour", "brain")

    data_set = pd.read_csv(csv_file_path)
    results = []

    for ind in data_set.index:
        patient_id = str(data_set['Patient_ID'][ind]).strip()
        mask_name  = str(data_set['Mask_Name'][ind]).strip()

        print(f"Processing case {ind} {patient_id} {mask_name}")

        # skip correto (afeta o ciclo principal)
        if any(w in mask_name.lower() for w in skip_words):
            print(f"Skipping {patient_id} ({mask_name}).")
            continue

        mr_dir = Path(str(data_set['MR_Treatment_Path'][ind]).strip())
        mr_vol_name = str(data_set['MR_Treatment_Volume'][ind]).strip()
        mask_path = Path(str(os.path.join(
            str(data_set['MR_Treatment_Path'][ind]).strip(),
            str(data_set['Treatment_ROI_Mask'][ind]).strip()
        )))

        mr_vol = resolve_registered_volume(mr_dir, mr_vol_name)

        print("mask_path:", repr(str(mask_path)))
        print("mr_vol   :", repr(str(mr_vol)) if mr_vol else None)

        if not mr_vol or not mr_vol.exists():
            print(" MR treatment volume not found. Skipping.")
            continue
        if not mask_path.exists():
            print(" Mask not found. Skipping.")
            continue

        try:
            # resample + binarização (0/255)
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol, binary_value=255)

            # máscara vazia?
            if np.sum(sitk.GetArrayFromImage(mask_resampled)) == 0:
                print("Mask empty after resampling. Skipping.")
                continue

            feature_vector = OrderedDict([('Mask Name', mask_name), ('Patient ID', patient_id)])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)
            print(" * Number of features extracted:", len(feature_result))

        except Exception as e:
            print(f" Error extracting features para {patient_id}: {e}")
            continue

    if not results:
        print("No result extracted. Verify everything!")
        return

    print("Done processing. Writing to file...")
    data_frame = pd.DataFrame.from_dict(results[0], orient="index")
    for idx, result_dict in enumerate(results):
        if idx == 0:
            continue
        df = pd.DataFrame.from_dict(result_dict, orient="index")
        data_frame = pd.concat([data_frame, df], axis=1)

    data_frame_T = data_frame.T
    output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_Treatment_ROI_Individual.csv'
    data_frame_T.to_csv(output_csv_path, index=False)
    print("Done! Saved:", output_csv_path)

f_extraction()

Processing case 0 Patient_61 mask_PTV-3x8
mask_path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/mask_PTV-3x8_treat.nii.gz'
mr_vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/P61_RM_1_TRATAMENTO_Registered_1.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 * Number of features extracted: 1338
Processing case 1 Patient_61 mask_PTV-3x8-1cm
mask_path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/mask_PTV-3x8-1cm_treat.nii.gz'
mr_vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/P61_RM_1_TRATAMENTO_Registered_1.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 * Number of features extracted: 1338
Processing case 2 Patient_61 mask_Tumor
mask_path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/mask_Tumor_treat.nii.gz'
mr_vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO/P61_RM_1_TRATAMENTO_Registered_1.nii.gz'


...

 * Number of features extracted: 1338
Processing case 244 Patient_85 mask_Tumor
mask_path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_TRATAMENTO/mask_Tumor_treat.nii.gz'
mr_vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_TRATAMENTO/P85_RM_1_TRATAMENTO_Registered_1.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 * Number of features extracted: 1338
Processing case 245 Patient_85 mask_Outer-Contour
Skipping Patient_85 (mask_Outer-Contour).
Processing case 246 Patient_85 mask_PTV-3x9
mask_path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_TRATAMENTO/mask_PTV-3x9_treat.nii.gz'
mr_vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_TRATAMENTO/P85_RM_1_TRATAMENTO_Registered_1.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 * Number of features extracted: 1338
Done processing. Writing to file...
Done! Saved: /notebooks/disk2/CSV_Dataset/02_Features_Follow_Up_ROI_Individual.csv


### Extract Features from MRI Follow_Up Image using Individual ROI Mask

In [41]:
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_ROI_Individual.csv'
ROOT = Path("/notebooks/disk2/RT_Dataset_2/06_Processed_Patients")

# helpers
def resample_mask_to_image(mask_path, image_path, binary_value=255):
    """Resample mask to match the MRI volume and binarize to 0 / binary_value."""
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path))
    mask_resampled = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, mask.GetPixelID()
    )
    mask_np = sitk.GetArrayFromImage(mask_resampled)
    mask_np[mask_np != 0] = binary_value
    mask_resampled = sitk.GetImageFromArray(mask_np)
    mask_resampled.SetOrigin(image.GetOrigin())
    mask_resampled.SetSpacing(image.GetSpacing())
    mask_resampled.SetDirection(image.GetDirection())
    return mask_resampled, image

def resolve_registered_volume(mr_dir: Path, fname: str) -> Path | None:
    """Use CSV name; if missing, try *_Registered_1.nii.gz, else the first *_Registered_*.nii.gz."""
    mr_dir = Path(str(mr_dir).strip())
    fname  = str(fname).strip()

    direct = mr_dir / fname
    if direct.exists():
        return direct

    if fname.endswith("Registered.nii.gz"):
        alt = mr_dir / fname.replace("Registered.nii.gz", "Registered_1.nii.gz")
        if alt.exists():
            return alt

    candidates = sorted(mr_dir.glob("*Registered_*.nii.gz"))
    return candidates[0] if candidates else None

def fix_followup_dir(raw_dir: str) -> str:
    """Ensure FOLLOW_UP dir includes 'RM' (e.g., P61_1_FOLLOW_UP -> P61_RM_1_FOLLOW_UP)."""
    s = str(raw_dir).strip()
    if "_FOLLOW_UP" in s and "RM" not in s:
        # conservative replacement
        s = s.replace("_1_FOLLOW_UP", "_RM_1_FOLLOW_UP")
    return s

def fix_followup_mask_name(raw_name: str) -> str:
    """Ensure mask filename has '_treat_fu' (not just '_fu'), matching your folder."""
    n = str(raw_name).strip()
    if n.endswith("_fu.nii.gz") and "_treat_fu.nii.gz" not in n:
        n = n.replace("_fu.nii.gz", "_treat_fu.nii.gz")
    return n

# main function 
def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    skip_words = ("skull", "outer contour", "outer-contour", "brain")

    data_set = pd.read_csv(csv_file_path)
    results = []

    for ind in data_set.index:
        patient_id = str(data_set['Patient_ID'][ind]).strip()
        mask_name  = str(data_set['Mask_Name'][ind]).strip()

        print(f"Processing case {ind} {patient_id} {mask_name}")

        # skip masks by keyword
        if any(w in mask_name.lower() for w in skip_words):
            print(f"Skipping {patient_id} ({mask_name}).")
            continue

        # ---- build absolute FOLLOW-UP dir (fix missing 'RM') ----
        fu_dir_rel = fix_followup_dir(data_set['MR_Follow_Up_Path'][ind])
        fu_dir = ROOT / patient_id / fu_dir_rel
        #print(fu_dir_rel)
       
        # ---- MR volume: Registered -> Registered_1 if needed ----
        mr_vol_name = str(data_set['MR_Follow_Up_Volume'][ind]).strip()
        mr_vol = resolve_registered_volume(fu_dir, mr_vol_name)

        # ---- MASK path: ensure '_treat_fu' suffix ----
        mask_name_csv = str(data_set['Follow_Up_ROI_Mask'][ind]).strip()
        mask_fname = fix_followup_mask_name(mask_name_csv)
        mask_path = fu_dir / mask_fname

        print("FU dir   :", str(fu_dir))
        print("MR vol   :", repr(str(mr_vol)) if mr_vol else None)
        print("Mask path:", repr(str(mask_path)))

        if not mr_vol or not mr_vol.exists():
            print("MR follow-up volume not found. Skipping.")
            continue
        if not mask_path.exists():
            print("Mask not found. Skipping.")
            continue

        try:
            # resample + binarize (0/255)
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol, binary_value=255)

            # empty mask check
            if np.sum(sitk.GetArrayFromImage(mask_resampled)) == 0:
                print("Mask empty after resampling. Skipping.")
                continue

            # extract radiomics
            feature_vector = OrderedDict([('Mask Name', mask_name), ('Patient ID', patient_id)])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)

            print("Number of features extracted:", len(feature_result))

        except Exception as e:
            print(f"Error extracting features for {patient_id}: {e}")
            continue

    if not results:
        print("No result extracted. Check paths/filenames in CSV vs. disk.")
        return

    print("Done processing. Writing to file...")
    data_frame = pd.DataFrame.from_dict(results[0], orient="index")
    for idx, result_dict in enumerate(results):
        if idx == 0:
            continue
        df = pd.DataFrame.from_dict(result_dict, orient="index")
        data_frame = pd.concat([data_frame, df], axis=1)

    data_frame_T = data_frame.T
    output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_Follow_Up_ROI_Individual.csv'
    data_frame_T.to_csv(output_csv_path, index=False)
    print("Done! Saved:", output_csv_path)
 
f_extraction()

Processing case 0 Patient_61 mask_PTV-3x8
FU dir   : /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
MR vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/P61_RM_1_FOLLOW_UP_Registered_1.nii.gz'
Mask path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/mask_PTV-3x8_treat_fu.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Number of features extracted: 1338
Processing case 1 Patient_61 mask_PTV-3x8-1cm
FU dir   : /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
MR vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/P61_RM_1_FOLLOW_UP_Registered_1.nii.gz'
Mask path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/mask_PTV-3x8-1cm_treat_fu.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Number of features extracted: 1338
Processing case 2 Patient_61 mask_Tumor
FU dir   : /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
MR vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/P61_RM_1_FOLLOW_UP_Registered_1.nii.gz'
Mask path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP/mask_Tumor_treat_fu.nii.gz'


...

Number of features extracted: 1338
Processing case 244 Patient_85 mask_Tumor
FU dir   : /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP
MR vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP/P85_RM_1_FOLLOW_UP_Registered_1.nii.gz'
Mask path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP/mask_Tumor_treat_fu.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Number of features extracted: 1338
Processing case 245 Patient_85 mask_Outer-Contour
Skipping Patient_85 (mask_Outer-Contour).
Processing case 246 Patient_85 mask_PTV-3x9
FU dir   : /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP
MR vol   : '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP/P85_RM_1_FOLLOW_UP_Registered_1.nii.gz'
Mask path: '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_85/P85_RM_1_FOLLOW_UP/mask_PTV-3x9_treat_fu.nii.gz'


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Number of features extracted: 1338
Done processing. Writing to file...
Done! Saved: /notebooks/disk2/CSV_Dataset/02_Features_Follow_Up_ROI_Individual.csv


### Extract Features from CT Treatment Image using Individual ROI Mask

In [ ]:
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_ROI_Individual.csv'
ROOT = Path("/notebooks/disk2/RT_Dataset_2/06_Processed_Patients")

# helpers
def resample_mask_to_image(mask_path, image_path, binary_value=255):
    """Resample mask to match the MRI volume and binarize to 0 / binary_value."""
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path))
    mask_resampled = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, mask.GetPixelID()
    )
    mask_np = sitk.GetArrayFromImage(mask_resampled)
    mask_np[mask_np != 0] = binary_value
    mask_resampled = sitk.GetImageFromArray(mask_np)
    mask_resampled.SetOrigin(image.GetOrigin())
    mask_resampled.SetSpacing(image.GetSpacing())
    mask_resampled.SetDirection(image.GetDirection())
    return mask_resampled, image


def fix_ct_mask_name(raw_name: str) -> str:
    """Ensure mask filename has '_treat_fu' (not just '_fu'), matching your folder."""
    n = str(raw_name).strip()
    if n.endswith("_ct.nii.gz") and "_treat_ct.nii.gz" not in n:
        n = n.replace("_ct.nii.gz", "_treat_ct.nii.gz")
    return n

# main function 
def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    skip_words = ("skull", "outer contour", "outer-contour", "brain")

    data_set = pd.read_csv(csv_file_path)
    results = []

    for ind in data_set.index:
        patient_id = str(data_set['Patient_ID'][ind]).strip()
        mask_name  = str(data_set['Mask_Name'][ind]).strip()

        print(f"Processing case {ind} {patient_id} {mask_name}")

        # skip masks by keyword
        if any(w in mask_name.lower() for w in skip_words):
            print(f"Skipping {patient_id} ({mask_name}).")
            continue

        # ---- MASK path: ensure '_treat_fu' suffix ----
        mask_name_csv = str(data_set['CT_ROI_Mask'][ind]).strip()
        mask_fname = fix_ct_mask_name(mask_name_csv)
        mask_path = fu_dir / mask_fname

        print("CT dir   :", str(fu_dir))
        print("MR vol   :", repr(str(mr_vol)) if mr_vol else None)
      
        if not mr_vol or not mr_vol.exists():
            print("MR CT volume not found. Skipping.")
            continue
        if not mask_path.exists():
            print("Mask not found. Skipping.")
            continue

        try:
            # resample + binarize (0/255)
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol, binary_value=255)

            # empty mask check
            if np.sum(sitk.GetArrayFromImage(mask_resampled)) == 0:
                print("Mask empty after resampling. Skipping.")
                continue

            # extract radiomics
            feature_vector = OrderedDict([('Mask Name', mask_name), ('Patient ID', patient_id)])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)

            print("Number of features extracted:", len(feature_result))

     except Exception as e:
            print(f"Error extracting features for {patient_id}: {e}")
            continue

    if not results:
        print("No result extracted. Check paths/filenames in CSV vs. disk.")
        return

    print("Done processing. Writing to file...")
    data_frame = pd.DataFrame.from_dict(results[0], orient="index")
    for idx, result_dict in enumerate(results):
        if idx == 0:
            continue
        df = pd.DataFrame.from_dict(result_dict, orient="index")
        data_frame = pd.concat([data_frame, df], axis=1)

    data_frame_T = data_frame.T
    output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_CT_ROI_Individual.csv'
    data_frame_T.to_csv(output_csv_path, index=False)
    print("Done! Saved:", output_csv_path)
 
f_extraction()

### Extract Features from MRI Treatment Image using Isodose Mask

In [4]:
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_Isodose.csv'
output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_Treatment_Isodose.csv'

from pathlib import Path

def mask_has_voxels(mask_image):
    """Check if the mask has any non-zero voxels."""
    arr = sitk.GetArrayFromImage(mask_image)
    return np.count_nonzero(arr) > 0

def resample_mask_to_image(mask_path, image_path):
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path), sitk.sitkUInt8)

    mask_r = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, sitk.sitkUInt8
    )
    return mask_r, image

def find_isodose_mask(patient_id: str, roi_label: str):
    # extract patient number (e.g., "54" from "Patient_54")
    number = patient_id.split("_")[1]
    
    # correct CT folder path with "P{number}"
    base_dir = Path(f"/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/{patient_id}/P{number}_CT_1_TRATAMENTO")
    if not base_dir.exists():
        return None
    
    for file in base_dir.iterdir():
        if file.is_file() and "Resampled" not in file.name and "Isodose_Volume" in file.name:
            if roi_label in file.name:
                return file
    
    return None         
    
def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    extractor.settings['label'] = 1

    df = pd.read_csv(csv_file_path)
    results=[]
    n_total = len(df)
    n_done = n_skipped_empty = n_skipped_missing = 0

    for ind in df.index:
        patient_id = str(df.at[ind, 'Patient_ID']).strip()
        roi_label = str(df.at[ind, 'Mask_Name']).strip().replace(".nii.gz","").replace(".nii","")

        mr_dir = Path(str(df.at[ind, 'MR_Treatment_Path']).strip())
        mr_vol  = mr_dir / str(df.at[ind, 'MR_Treatment_Volume']).strip()

        # NEW: look up isodose mask from folder
        mask_path = find_isodose_mask(patient_id, roi_label)

        print(f"[{ind}/{n_total}] Patient: {patient_id} | ROI: {roi_label}")
        if not mr_vol.exists():
            print(f" MR volume not found: {mr_vol}")
            n_skipped_missing += 1
            continue
        if not mask_path or not mask_path.exists():
            print(f" Isodose mask not found for {patient_id}")
            n_skipped_missing += 1
            continue

        try:
            mask_image = sitk.ReadImage(str(mask_path))
        except Exception as e:
            print(f" Error reading mask '{mask_path}': {e}")
            n_skipped_missing += 1
            continue

        if not mask_has_voxels(mask_image):
            print(" Mask is empty. Skipping.")
            n_skipped_empty += 1
            continue

        try:
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol)
            if not mask_has_voxels(mask_resampled):
                print(" Mask empty after resampling. Skipping.")
                n_skipped_empty += 1
                continue
        except Exception as e:
            print(f" Error during resampling: {e}")
            n_skipped_missing += 1
            continue

        try:
            feature_vector = OrderedDict([
                ('Patient ID', patient_id),
                ('Mask Name', roi_label),
            ])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)
            n_done += 1
            print(f" Features extracted: {len(feature_result)}")
        except Exception as e:
            print(f" Error extracting features: {e}")
            continue

    if not results:
        print("No features extracted.")
        return

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv_path, index=False)
    print("Done!")
    print(f"Saved: {output_csv_path}")
    print(f"Processed: {n_done} | Skipped empty: {n_skipped_empty} | Skipped missing: {n_skipped_missing}")

f_extraction()

[0/175] Patient: Patient_54 | ROI: mask_PTV-frontal-D-17Gy


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[1/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-anterior


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[2/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-post-18Gy


...

 Features extracted: 1338
[171/175] Patient: Patient_95 | ROI: mask_GTV-temporal-E
 Mask is empty. Skipping.
[172/175] Patient: Patient_95 | ROI: mask_GTV-temporal-d


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[173/175] Patient: Patient_96 | ROI: mask_FRONTAL-D


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[174/175] Patient: Patient_96 | ROI: mask_GTV-PARIETAL-D
 Mask is empty. Skipping.
Done!
Saved: /notebooks/disk2/CSV_Dataset/02_Features_Treatment_Isodose.csv
Processed: 103 | Skipped empty: 72 | Skipped missing: 0


### Extract Features from MRI Follow_Up Image using Isodose Mask

In [7]:
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_Isodose.csv'
output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_Follow_Up_Isodose.csv'

from pathlib import Path

def mask_has_voxels(mask_image):
    """Check if the mask has any non-zero voxels."""
    arr = sitk.GetArrayFromImage(mask_image)
    return np.count_nonzero(arr) > 0

def resample_mask_to_image(mask_path, image_path):
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path), sitk.sitkUInt8)

    mask_r = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, sitk.sitkUInt8
    )
    return mask_r, image

def find_isodose_mask(patient_id: str, roi_label: str):
    # extract patient number (e.g., "54" from "Patient_54")
    number = patient_id.split("_")[1]
    
    # correct CT folder path with "P{number}"
    base_dir = Path(f"/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/{patient_id}/P{number}_CT_1_TRATAMENTO")
    if not base_dir.exists():
        return None
    
    for file in base_dir.iterdir():
        if file.is_file() and "Resampled" not in file.name and "Isodose_Volume" in file.name:
            if roi_label in file.name:
                return file
    
    return None         
    
def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    extractor.settings['label'] = 1

    df = pd.read_csv(csv_file_path)
    results=[]
    n_total = len(df)
    n_done = n_skipped_empty = n_skipped_missing = 0

    for ind in df.index:
        patient_id = str(df.at[ind, 'Patient_ID']).strip()
        roi_label = str(df.at[ind, 'Mask_Name']).strip().replace(".nii.gz","").replace(".nii","")

        mr_dir = Path(str(df.at[ind, 'Follow_Up_Path']).strip())
        mr_vol  = mr_dir / str(df.at[ind, 'Follow_Up_Volume']).strip()

        # NEW: look up isodose mask from folder
        mask_path = find_isodose_mask(patient_id, roi_label)


        print(f"[{ind}/{n_total}] Patient: {patient_id} | ROI: {roi_label}")
        if not mr_vol.exists():
            print(f" MR volume not found: {mr_vol}")
            n_skipped_missing += 1
            continue
        if not mask_path or not mask_path.exists():
            print(f" Isodose mask not found for {patient_id}")
            n_skipped_missing += 1
            continue

        try:
            mask_image = sitk.ReadImage(str(mask_path))
        except Exception as e:
            print(f" Error reading mask '{mask_path}': {e}")
            n_skipped_missing += 1
            continue

        if not mask_has_voxels(mask_image):
            print(" Mask is empty. Skipping.")
            n_skipped_empty += 1
            continue

        try:
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol)
            if not mask_has_voxels(mask_resampled):
                print(" Mask empty after resampling. Skipping.")
                n_skipped_empty += 1
                continue
        except Exception as e:
            print(f" Error during resampling: {e}")
            n_skipped_missing += 1
            continue

        try:
            feature_vector = OrderedDict([
                ('Patient ID', patient_id),
                ('Mask Name', roi_label),
            ])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)
            n_done += 1
            print(f" Features extracted: {len(feature_result)}")
        except Exception as e:
            print(f" Error extracting features: {e}")
            continue

    if not results:
        print("No features extracted.")
        return

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv_path, index=False)
    print("Done!")
    print(f"Saved: {output_csv_path}")
    print(f"Processed: {n_done} | Skipped empty: {n_skipped_empty} | Skipped missing: {n_skipped_missing}")

f_extraction()

[0/175] Patient: Patient_54 | ROI: mask_PTV-frontal-D-17Gy


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[1/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-anterior


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[2/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-post-18Gy


...

 Features extracted: 1338
[171/175] Patient: Patient_95 | ROI: mask_GTV-temporal-E
 Mask is empty. Skipping.
[172/175] Patient: Patient_95 | ROI: mask_GTV-temporal-d


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[173/175] Patient: Patient_96 | ROI: mask_FRONTAL-D


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[174/175] Patient: Patient_96 | ROI: mask_GTV-PARIETAL-D
 Mask is empty. Skipping.
Done!
Saved: /notebooks/disk2/CSV_Dataset/02_Features_Follow_Up_Isodose.csv
Processed: 103 | Skipped empty: 72 | Skipped missing: 0


### Extract Features from CT Treatment Image using Isodose Mask

In [ ]:
# Extract Features from CT Treatment Image using Isodose Mask
csv_file_path = '/notebooks/disk2/CSV_Dataset/01_Isodose.csv'
output_csv_path = '/notebooks/disk2/CSV_Dataset/02_Features_CT_Isodose.csv'

from pathlib import Path

def mask_has_voxels(mask_image):
    """Check if the mask has any non-zero voxels."""
    arr = sitk.GetArrayFromImage(mask_image)
    return np.count_nonzero(arr) > 0

def resample_mask_to_image(mask_path, image_path):
    image = sitk.ReadImage(str(image_path))
    mask  = sitk.ReadImage(str(mask_path), sitk.sitkUInt8)

    mask_r = sitk.Resample(
        mask, image, sitk.Transform(),
        sitk.sitkNearestNeighbor, 0, sitk.sitkUInt8
    )
    return mask_r, image

def find_isodose_mask(patient_id: str, roi_label: str):
    # extract patient number (e.g., "54" from "Patient_54")
    number = patient_id.split("_")[1]
    
    # correct CT folder path with "P{number}"
    base_dir = Path(f"/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/{patient_id}/P{number}_CT_1_TRATAMENTO")
    if not base_dir.exists():
        return None
    
    for file in base_dir.iterdir():
        if file.is_file() and "Resampled" not in file.name and "Isodose_Volume" in file.name:
            if roi_label in file.name:
                return file
    
    return None         
    
def f_extraction():
    param_path = '/notebooks/disk2/Notebooks/Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(param_path)
    extractor.settings['label'] = 1

    df = pd.read_csv(csv_file_path)
    results=[]
    n_total = len(df)
    n_done = n_skipped_empty = n_skipped_missing = 0

    for ind in df.index:
        patient_id = str(df.at[ind, 'Patient_ID']).strip()
        roi_label = str(df.at[ind, 'Mask_Name']).strip().replace(".nii.gz","").replace(".nii","")

        mr_dir = Path(str(df.at[ind, 'CT_Treatment_Path']).strip())
        mr_vol  = mr_dir / str(df.at[ind, 'CT_Treatment_Volume']).strip()

        # NEW: look up isodose mask from folder
        mask_path = find_isodose_mask(patient_id, roi_label)


        print(f"[{ind}/{n_total}] Patient: {patient_id} | ROI: {roi_label}")
        if not mr_vol.exists():
            print(f" MR volume not found: {mr_vol}")
            n_skipped_missing += 1
            continue
        if not mask_path or not mask_path.exists():
            print(f" Isodose mask not found for {patient_id}")
            n_skipped_missing += 1
            continue

        try:
            mask_image = sitk.ReadImage(str(mask_path))
        except Exception as e:
            print(f" Error reading mask '{mask_path}': {e}")
            n_skipped_missing += 1
            continue

        if not mask_has_voxels(mask_image):
            print(" Mask is empty. Skipping.")
            n_skipped_empty += 1
            continue

        try:
            mask_resampled, image = resample_mask_to_image(mask_path, mr_vol)
            if not mask_has_voxels(mask_resampled):
                print(" Mask empty after resampling. Skipping.")
                n_skipped_empty += 1
                continue
        except Exception as e:
            print(f" Error during resampling: {e}")
            n_skipped_missing += 1
            continue

        try:
            feature_vector = OrderedDict([
                ('Patient ID', patient_id),
                ('Mask Name', roi_label),
            ])
            feature_result = extractor.execute(image, mask_resampled)
            feature_vector.update(feature_result)
            results.append(feature_vector)
            n_done += 1
            print(f" Features extracted: {len(feature_result)}")
        except Exception as e:
            print(f" Error extracting features: {e}")
            continue

    if not results:
        print("No features extracted.")
        return

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv_path, index=False)
    print("Done!")
    print(f"Saved: {output_csv_path}")
    print(f"Processed: {n_done} | Skipped empty: {n_skipped_empty} | Skipped missing: {n_skipped_missing}")

f_extraction()

[0/175] Patient: Patient_54 | ROI: mask_PTV-frontal-D-17Gy


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[1/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-anterior


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[2/175] Patient: Patient_54 | ROI: mask_PTV-frontal-E-post-18Gy


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[162/175] Patient: Patient_91 | ROI: mask_GTV-cerebelo-E


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

 Features extracted: 1338
[163/175] Patient: Patient_91 | ROI: mask_GTV-occiptal-E-2
 Mask is empty. Skipping.
[164/175] Patient: Patient_91 | ROI: mask_GTV-occiptal-E
 Mask is empty. Skipping.
[165/175] Patient: Patient_91 | ROI: mask_GTV-parietal-post


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
